# Training the memory node networks

The following document outlines generating the weights for the networks used in the memory node of the framework.

## Pre-amble

In [1]:
# Import necessary libraries
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
import time
import matplotlib.pyplot as plt
import re
import pickle
import tensorflow as tf
import os

In [2]:
# Define class for manipulating DEP generated and stored C matrices from file
# Allows for matrix to be loaded from file created by dep_gui
# A reduced version of these matrices are used as most muscles are inactive obtained via reduced_matrix()
class matrix_expansion:
    
    def __init__(self,active_motors):
        self.active_motors = active_motors
        active_sensors = np.array(active_motors*2)
        active_sensors[len(active_motors):] += 14
        self.active_sensors = active_sensors
        self.shape = ()
        
    def load_from_file(self,filename):
        f = open(filename,"r")
        matrix = f.read()
        f.close()
        matrix = re.split(",NEW_ROW,",matrix)
        matrix.pop()
        matrix = np.array([np.array(re.split(",", row)).astype(np.float) for row in matrix])
        self.shape = matrix.shape
        return matrix
        
    def reduced_matrix(self,matrix):
        matrix = matrix[:,self.active_sensors][self.active_motors]
        return matrix
    
    def expanded_matrix(self,reduced_matrix):
        matrix = np.zeros(self.shape)
        flat = reduced_matrix.flatten()
        matrix = np.zeros((14,28))
        k = 0
        for i in active_motors:
            for j in active_sensors:
                matrix[i,j] = flat[k]
                k += 1
        return matrix

In [3]:
# Encoding

# HTM SDR Scalar Encoder
# Input: Scalar
# Parameters: n - number of units, w - bits used to represent signal (width), b - buckets (i.e. resolution), 
#             min - minimum value of input (inclusive), max - maximum input value (inclusive)
class scalar_sdr:
    
    def __init__(self, b, w, min_, max_, shape=(1,1), neg=True):
        if type(b) != int or type(w) != int or type(min_) != float or type(max_) != float:
            raise TypeError("b - buckets must be int, w - width must be int, min_ must be float and max_ must be float")
        self.b = b # must be int
        self.w = w # must be int
        self.min = min_ # must be float
        self.max = max_ # must be float
        self.n = b+w-1 # number of units for encoding
        self.ndarray_shape = shape
        self.nodes = self.n*reduce(lambda x, y: x*y, self.ndarray_shape)
        self.neg = neg
        
    def encode(self,input_):
        if input_ > self.max or input_ < self.min:
            raise ValueError("Input outside encoder range!")
        if type(input_) != float:
            raise TypeError("Input must be float!")
        if self.neg:
            output = np.zeros(self.n)-1
        else:
            output = np.zeros(self.n)
        index = int((input_-self.min)/(self.max-self.min)*self.b)
        output[index:index+self.w] = 1
        return output
    
    def encode_ndarray(self,input_):
        if input_.shape != self.ndarray_shape:
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        output = []
        for i in np.nditer(input_, order='K'):
            output.append(self.encode(float(i)))
        return np.array(output)
    '''
    def decode(self,input_):
        if len(input_) != self.n: # or len(np.nonzero(input_+1)[0]) != self.w: <-- Can't have since the network is not guaranteed to produce this by any means!!!
            raise TypeError("Input does not correspond to encoder encoded data!")
        # output = np.nonzero(input_+1)[0][0]/float(self.b)*(self.max-self.min)+self.min <-- This doesn't work really since bits can randomly fire, taking the average is a more reasonable decoding
        median = np.median(np.nonzero(input_+1)[0])            
        try:
            output = int(median-float(self.w)/2.0)/float(self.b)*(self.max-self.min)+self.min # i.e. figure out center (median more outlier resistant than mean) and subtract width/2
        except ValueError:
            output = None
        return output
    '''
    def decode(self,input_):
        if len(input_) != self.n: 
            raise TypeError("Input length does not match encoder length!")
        if len(np.nonzero(input_+1)[0]) == 0:
            return np.nan
        max_ = 0
        output = 0.0
        for i in range(self.b):
            x = np.zeros(self.n)-1
            x[i:i+self.w] = 1
            if x.shape != input_.shape:
                input_ = input_.reshape(x.shape)
            score = np.inner(x,input_)
            if score > max_:
                max_ = score
                output = float(i)/float(self.b)*(self.max-self.min)+self.min
        return output
            
    def decode_ndarray(self,input_):
        if input_.shape != (reduce(lambda x, y: x*y, self.ndarray_shape)*self.n,): 
            raise ValueError("Input dimensions do not match specified encoder dimensions!")
        input_ = input_.reshape(self.ndarray_shape+(self.n,))
        output = []
        for i in np.ndindex(self.ndarray_shape):
            output.append(self.decode(input_[i]))
        output = np.array(output).reshape(self.ndarray_shape)
        return output
    
    def set_ndarray_shape(self,shape):
        if type(shape) != tuple:
            raise TypeError("Must provide tuple of array dimensions!")
        self.ndarray_shape = shape

## Linear Associative Memory (LAM) behavior weights

This section generates the weights for the LAM network used in the memory node from four desired behaviors: i) zero--no behavior; ii) front back; iii) front side; and iv) side down. For more information on each of the behaviors you can refer to the thesis or experiment with them for yourself with the GUI!

In [31]:
# LAM Class
class LAM:
    def __init__(self,shape,weights=None):
        self.shape = shape
        try:
            if weights == None:
                self.weights = np.zeros(shape)
        except ValueError:
            self.weights = weights
    
    def store(self,input,output):
        dW = np.outer(input,output)
        self.weights += dW
        
    def recall(self,input,threshold=0,print_=False):
        output = input.dot(self.weights)-threshold
        if print_ == True:
            print output
        output[output > 0] = 1
        output[output < 0] = 0
        return output

    def save_weights(self,filename):
        np.save(filename, self.weights)
        
    def load_weights(self,filename):
        weights = np.load(filename)
        if weights.shape == self.shape:
            self.weights = weights
        else:
            raise ValueError("Dimensions of specified weight array does not match network weight dimensions!")

In [32]:
# Brain encoder
# Buckets = 1000, width = 21, range = 1000
brain_encoder = scalar_sdr(1000,21,0.,1000.,neg=False)

# We have four behaviors that we want to store in the network
names = ["zero","fb","fs","sd"]

# For each of these behaviors lets create an integer ID
# We need the ids to have no overlap and thus they are seperated by the width of the brain_encoder
brain_ids = [float(i)*brain_encoder.w for i in range(len(names))]

# This is just a dictionary associating each of the labels to the integer ID
behaviors = dict(zip(names,brain_ids))

In [33]:
# Get matrix data for desired behaviors from matrix files generated by GUI
# Note: need to define desired behaviors from set of files in /{HOME_ENVIRONMENT}/dep_matrices
# The matrices are located in the "home" directory if new behaviors are generated with the GUI 
# and/or the provided installation instructions are followed

home = os.getenv("HOME")

active_motors = [1,3,4,5,10,12]
expander = matrix_expansion(active_motors)

# Front back
filename = home+"/dep_matrices/front_back.dep"
fb_matrix = expander.load_from_file(filename)
fb_reduced = expander.reduced_matrix(fb_matrix)
#fb_expanded = expander.expanded_matrix(fb_reduced)

# Front side
filename = home+"/dep_matrices/front_side.dep"
fs_matrix = expander.load_from_file(filename)
fs_reduced = expander.reduced_matrix(fs_matrix)
#fs_expanded = expander.expanded_matrix(fs_reduced)

# Side down
filename = home+"/dep_matrices/side_down.dep"
sd_matrix = expander.load_from_file(filename)
sd_reduced = expander.reduced_matrix(sd_matrix)
#sd_expanded = expander.expanded_matrix(sd_reduced)

# Zero
zero_matrix = np.zeros(fb_matrix.shape)
zero_reduced = np.zeros(fb_reduced.shape)

#matrices = {"fb": fb_matrix, "fs": fs_matrix, "sd": sd_matrix, "zero": zero_matrix}
matrices = {"fb": fb_reduced, "fs": fs_reduced, "sd": sd_reduced, "zero": zero_reduced}

In [34]:
# Storing the ID and matrix pairs

active_motors = [1,3,4,5,10,12]
n = len(active_motors)
matrix_shape = (n,n*2)
matrix_encoder = scalar_sdr(100,21,-0.25,0.25,matrix_shape,neg=False)
shape = (brain_encoder.n,matrix_encoder.n*reduce(lambda x, y: x*y, matrix_encoder.ndarray_shape))

matrix_lam = LAM(shape)
for id_ in behaviors:    
    brain_sig = brain_encoder.encode(behaviors[id_])
    matrix = matrix_encoder.encode_ndarray(matrices[id_])
    matrix_lam.store(brain_sig,matrix)

In [35]:
mem_pickle = {"brain_encoder": brain_encoder, "matrix_encoder": matrix_encoder, "lam": matrix_lam, "brain_id_to_behv_id":dict(zip(brain_ids,names))}
pickle_out = open("data/new/mem.pickle","wb") # note: another directory is used to not overwrite the previous files
pickle.dump(mem_pickle, pickle_out)
pickle_out.close()

## Trigger network weights

This section generates the weights for the trigger network. Notably, a seperate set of weights is used for each behavior and stored as a dictionary. This could just as well be implemented using an associative memory. 

In [71]:
# load recordings of base behaviors
fb = pickle.load(open("data/bases/fb.pickle","rb"))
fs = pickle.load(open("data/bases/fs.pickle","rb"))
sd = pickle.load(open("data/bases/sd.pickle", "rb"))
zero = [np.zeros(fb[0].shape),np.zeros(fb[1].shape), np.zeros(fb[2].shape)]

# define a dictionary for convenience
bases = {"fb": fb, "fs": fs, "sd": sd, "zero": zero}

In [72]:
# active motors
active_motors = [1,3,4,5,10,12]

# define "optimal" transition points
# note: manually obtained
ts = [{},{}]
ts[0] = {'fb': 124, 'fs': 126, 'sd': 118, 'zero': 0}
ts[1] = {'fb': 62, 'fs': 63, 'sd': 59, 'zero': 0}

In [73]:
pos_encoder = scalar_sdr(7,1,-100000.0,100000.0,shape=(6,1),neg=False)

behvs = {}
for id_ in bases:
    behvs[id_] = {}
    behvs[id_]["weights_pos"] = []
    for i in active_motors:
        m_pos = np.zeros((pos_encoder.n))
        for j in range(2):
            pos = pos_encoder.encode(float(bases[id_][0][ts[j][id_]][i]))
            m_pos += pos # hebbian learning of weights with only direct connections
        behvs[id_]["weights_pos"].append(m_pos)
    behvs[id_]["weights_pos"] = np.array(behvs[id_]["weights_pos"]).reshape(1,6,pos_encoder.n,1)
behvs["pos_encoder"] = pos_encoder

In [75]:
pickle_out = open("data/new/neurons_pos.pickle","wb")  # note: another directory is used to not overwrite the previous files
pickle.dump(behvs, pickle_out)
pickle_out.close()

## Hopfield behavior weights

The implementation of the memory node does not use Hopfield as default since the performance is degraded. However, this may be used, and the weights can be obtained as follows.

In [4]:
# Implementation for asynchronous Hopfield neural network
# Note: memory capacity ≃ 0.14*nodes
class Hopfield_Neural_Network:
    def __init__(self,nodes,iterations=100,weights=None):
        self.nodes = nodes
        self.iterations = iterations
        try:
            if weights == None:
                self.weights = np.zeros((nodes,nodes))
        except ValueError:
            self.weights = weights
    
    def store(self,input):
        dW = np.outer(input,input)
        np.fill_diagonal(dW,0)
        self.weights += dW
        
    def recall(self,input,range_=None):
        # Can specify range of nodes to iterate over (i.e. nodes that are "input" may be known as correct)
        if type(range_) == tuple:
            a = range(range_[0],range_[1])
        else:
            a = self.nodes
        update_sequence = np.random.choice(a, self.iterations)
        for node in update_sequence:
            input[node] = np.sign(np.inner(input,self.weights[:,node]))
            if input[node] == 0: # this was missing
                input[node] = -1
        return input
    
    def setIter(self,iter_):
        self.iterations = iter_
    
    def save_weights(self,filename):
        np.save(filename, self.weights)
        
    def load_weights(self,filename):
        weights = np.load(filename)
        if weights.shape == (self.nodes, self.nodes):
            self.weights = weights
        else:
            raise ValueError("Dimensions of specified weight array does not match network weight dimensions!")

In [5]:
# Get matrix data for desired behaviors from matrix files generated by GUI
# Note: need to define desired behaviors from set of files in /{HOME_ENVIRONMENT}/dep_matrices
# The matrices are located in the "home" directory if new behaviors are generated with the GUI 
# and/or the provided installation instructions are followed

home = os.getenv("HOME")

active_motors = [1,3,4,5,10,12]
expander = matrix_expansion(active_motors)

# Front back
filename = home+"/dep_matrices/front_back.dep"
fb_matrix = expander.load_from_file(filename)
fb_reduced = expander.reduced_matrix(fb_matrix)
#fb_expanded = expander.expanded_matrix(fb_reduced)

# Front side
filename = home+"/dep_matrices/front_side.dep"
fs_matrix = expander.load_from_file(filename)
fs_reduced = expander.reduced_matrix(fs_matrix)
#fs_expanded = expander.expanded_matrix(fs_reduced)

# Side down
filename = home+"/dep_matrices/side_down.dep"
sd_matrix = expander.load_from_file(filename)
sd_reduced = expander.reduced_matrix(sd_matrix)
#sd_expanded = expander.expanded_matrix(sd_reduced)

# Zero
zero_matrix = np.zeros(fb_matrix.shape)
zero_reduced = np.zeros(fb_reduced.shape)

#matrices = {"fb": fb_matrix, "fs": fs_matrix, "sd": sd_matrix, "zero": zero_matrix}
matrices = {"fb": fb_reduced, "fs": fs_reduced, "sd": sd_reduced, "zero": zero_reduced}

In [6]:
brain_id = {"zero": 0.125, "fb": 0.375, "fs": 0.625, "sd": 0.875}

In [7]:
# Setup encoders
matrix_shape = (1,)
m_encoder = scalar_sdr(80*20,44*20,-0.25,0.25,matrix_shape)
b_encoder = scalar_sdr(92*20,40*20,0.0,1.0)

In [8]:
# Calculate number of nodes of hopfield net
nodes = m_encoder.nodes + b_encoder.nodes

In [ ]:
# i) Initialize hopfield network for each matrix value
# ii) Store data for each brain_id+matrix[brain_id] for given matrix index
nns = {}
for index in np.ndindex(matrices["zero"].shape):
    hnn = Hopfield_Neural_Network(nodes)
    for id_ in brain_id:
        data = np.array([])
        matrix = matrices[id_][index].reshape(matrix_shape)
        matrix = m_encoder.encode_ndarray(matrix)
        data = np.append(data,matrix.flatten())

        brain_sig = brain_id[id_]
        brain_sig = b_encoder.encode(brain_sig)
        data = np.append(data,brain_sig)

        hnn.store(data)

    nns[index] = hnn

In [ ]:
pickle_out = open("data/new/hnns_80.pickle","wb")
pickle.dump(nns, pickle_out)
pickle_out.close()